In [260]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, Point, Polygon
import os 
from pathlib import Path
import re

### Load Data

In [261]:
# get path to project root directory
project_root = Path.cwd().parents[0]

# build path to input folder
data_external = project_root/"data"/"external"

# build path to output folder
data_processed = project_root/"data"/"processed"

In [262]:
dtype_map = {
    "lat": "float64",
    "lon": "float64",
    "site_id": "string"
}

# load idp camps into a pandas dataframe
camps_df = pd.read_csv(data_external/"allEthiopiaSitesData_allTime_20231027.csv",low_memory = False)
camps_df.head(2)


,Unnamed: 0,SiteID,SiteName,SurveyCount,minSurveyDate,maxSurveyDate,minLong,maxLong,minLat,maxLat,...,R33_SiteName,R33_IsSiteOpen,R33_SiteStarted,R33_SiteType,R33_TotalHH,R33_TotIndivd,R33_SiteOpenDate,R33_SurveyDate,R33_DispReason,R33_DispReasonOther
0,0,AA101,Gotara,8,2018-05-09,2019-07-13,38.756062,38.756316,8.980032,8.980559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AF118,Andido,12,2018-04-30,2020-06-03,40.321054,40.324781,9.398847,9.401627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Quick Recon of Dataset

In [263]:
# define a new dataframe to contain important metadata about null values 
summary = pd.DataFrame({
    "dtype": camps_df.dtypes,
    "non_null": camps_df.count(),
    "nulls": camps_df.isna().sum(),
    "pct_null": camps_df.isna().mean() * 100,
})

# sort the features of the table by null values
summary.sort_values("pct_null", ascending=True)

,dtype,non_null,nulls,pct_null
Unnamed: 0,int64,4882,0,0.000000
SiteID,object,4882,0,0.000000
SiteName,object,4882,0,0.000000
SurveyCount,int64,4882,0,0.000000
minSurveyDate,object,4882,0,0.000000
...,...,...,...,...
R19_DispReasonOther,object,26,4856,99.467431
R18_DispReasonOther,object,22,4860,99.549365
R32_DispReasonOther,object,15,4867,99.692749
R31_DispReasonOther,object,15,4867,99.692749


In [264]:
# get lists of numeric and categorical column names
num_cols = camps_df.select_dtypes(include="number").columns
cat_cols = camps_df.select_dtypes(exclude="number").columns

# print the length of the column names
print(len(num_cols), len(cat_cols))

65 201


In [265]:
# generate summary stats of the 
camps_df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,4882.0,2440.500000,1409.456337,0.000000,1220.250000,2440.500000,3660.750000,4881.000000
SurveyCount,4882.0,7.114093,6.834800,1.000000,2.000000,5.000000,10.000000,23.000000
minLong,4882.0,39.001202,2.285182,33.221964,37.546877,38.873356,40.101531,47.479735
maxLong,4882.0,39.007205,2.286309,33.221964,37.548609,38.884107,40.104843,47.479735
minLat,4882.0,9.229325,2.994575,3.455774,6.789391,9.299709,11.321115,14.547139
...,...,...,...,...,...,...,...,...
R32_TotalHH,2472.0,234.495146,484.530993,20.000000,49.000000,108.000000,240.000000,10045.000000
R32_TotIndivd,2472.0,1271.543285,2765.324807,23.000000,244.000000,551.000000,1320.250000,60270.000000
R33_TotalHH,643.0,389.530327,657.652132,20.000000,70.000000,138.000000,356.000000,5661.000000
R33_TotIndivd,643.0,1589.110420,2828.550711,32.000000,218.500000,478.000000,1402.000000,21764.000000


In [266]:
cat_summary = pd.DataFrame({
    "unique_values": camps_df[cat_cols].nunique(),
    "top_value": camps_df[cat_cols].mode().iloc[0],
})

cat_summary.sort_values("unique_values", ascending=False)

,unique_values,top_value
SiteID,4882,AA101
SiteName,4487,Selam
R32_SiteName,2376,Addis Alem
R31_SiteName,2112,Kebele 02
R29_SiteName,2082,Addis Alem
...,...,...
R30_IsSiteOpen,2,Yes
R31_IsSiteOpen,2,Yes
R32_IsSiteOpen,2,Yes
R33_IsSiteOpen,2,Yes


In [267]:
# set pandas display options to view every row
pd.set_option("display.max_rows", None)

# view the column names of the geodataframe
pd.DataFrame({
    "column": camps_df.columns,
    "dtype": camps_df.dtypes.values
})

,column,dtype
0,Unnamed: 0,int64
1,SiteID,object
2,SiteName,object
3,SurveyCount,int64
4,minSurveyDate,object
5,maxSurveyDate,object
6,minLong,float64
7,maxLong,float64
8,minLat,float64
9,maxLat,float64


In [268]:
# reset the display options
pd.reset_option("display.max_rows")

### Reshape the dataset into LONG form for time series Animation

In [269]:
# define the important base cols
base_columns = ["SiteID", "SiteName", "RegionName", "meanLat", "meanLong", "minOpenDate", "maxOpenDate","maxSurveyDate"]
base_columns = [c for c in base_columns if c in camps_df.columns]

In [270]:
# use regular expression to get get a list of survey data columns
survey_pattern = re.compile(r"^(R\d{2})_") 
survey_prefixes = sorted({survey_pattern.match(c).group(1) for c in camps_df.columns if survey_pattern.match(c)})
survey_suffixes = ["SurveyDate", "TotIndivd", "TotalHH", "IsSiteOpen", "SiteType", "DispReason"]

survey_columns = []
for sp in survey_prefixes:
    for ss in survey_suffixes:
        col = f"{sp}_{ss}"
        if col in camps_df.columns:
            survey_columns.append(col)

print(survey_columns)

['R11_SurveyDate', 'R11_TotIndivd', 'R11_TotalHH', 'R11_IsSiteOpen', 'R11_SiteType', 'R11_DispReason', 'R12_SurveyDate', 'R12_TotIndivd', 'R12_TotalHH', 'R12_IsSiteOpen', 'R12_SiteType', 'R12_DispReason', 'R13_SurveyDate', 'R13_TotIndivd', 'R13_TotalHH', 'R13_IsSiteOpen', 'R13_SiteType', 'R13_DispReason', 'R14_SurveyDate', 'R14_TotIndivd', 'R14_TotalHH', 'R14_IsSiteOpen', 'R14_SiteType', 'R14_DispReason', 'R15_SurveyDate', 'R15_TotIndivd', 'R15_TotalHH', 'R15_IsSiteOpen', 'R15_SiteType', 'R15_DispReason', 'R16_SurveyDate', 'R16_TotIndivd', 'R16_TotalHH', 'R16_IsSiteOpen', 'R16_SiteType', 'R16_DispReason', 'R17_SurveyDate', 'R17_TotIndivd', 'R17_TotalHH', 'R17_IsSiteOpen', 'R17_SiteType', 'R17_DispReason', 'R18_SurveyDate', 'R18_TotIndivd', 'R18_TotalHH', 'R18_IsSiteOpen', 'R18_SiteType', 'R18_DispReason', 'R19_SurveyDate', 'R19_TotIndivd', 'R19_TotalHH', 'R19_IsSiteOpen', 'R19_SiteType', 'R19_DispReason', 'R20_SurveyDate', 'R20_TotIndivd', 'R20_TotalHH', 'R20_IsSiteOpen', 'R20_SiteType

In [271]:
# create the long table 
long = camps_df[base_columns + survey_columns].copy()


In [272]:
melted = long.melt(
    id_vars=base_columns,
    value_vars=survey_columns,
    var_name="survey_field",
    value_name="value"
)

melted

,SiteID,SiteName,RegionName,meanLat,meanLong,minOpenDate,maxOpenDate,maxSurveyDate,survey_field,value
0,AA101,Gotara,Addis Ababa,8.980229,38.756166,2017-09-22 00:00:00.000,2017-09-22,2019-07-13,R11_SurveyDate,2018-05-09 00:00:00
1,AF118,Andido,Afar,9.400856,40.323826,1900-01-09 09:37:00.118,2015-09-30,2020-06-03,R11_SurveyDate,2018-04-30 00:00:00
2,AF128,Bada Admorug,Afar,14.519210,40.116068,2016-06-10 00:00:00.000,2016-06-10,2023-08-05,R11_SurveyDate,2018-04-25 00:00:00
3,AF130,Gera,Afar,12.059997,41.925306,2015-01-20 00:00:00.000,2015-01-20,2023-08-30,R11_SurveyDate,2018-05-08 00:00:00
4,AF131d,Sodomta Drought,Afar,12.823153,40.989297,2014-04-21 00:00:00.000,2014-04-21,2021-08-24,R11_SurveyDate,2018-04-23 00:00:00
...,...,...,...,...,...,...,...,...,...,...
673711,TG1095,Laelay Adi Gebaro,Tigray,14.141199,38.177646,2020-12-31,2020-12-31,2023-09-02,R33_DispReason,NaN
673712,TG1096,May Seye,Tigray,14.108307,38.138678,2020-12-31,2020-12-31,2023-09-02,R33_DispReason,NaN
673713,TG1086,Simret,Tigray,14.224869,38.852089,2023-07-02,2023-07-02,2023-09-02,R33_DispReason,NaN
673714,TG1097,Geter Zeban Gedena,Tigray,14.326730,37.889691,2023-08-25,2023-08-25,2023-09-02,R33_DispReason,NaN


In [273]:
# survey_field like "R11_TotIndivd" -> survey="R11", field="TotIndivd"
melted[["survey", "field"]] = melted["survey_field"].str.split("_", n=1, expand=True)
melted = melted.drop(columns=["survey_field"])
melted

,SiteID,SiteName,RegionName,meanLat,meanLong,minOpenDate,maxOpenDate,maxSurveyDate,value,survey,field
0,AA101,Gotara,Addis Ababa,8.980229,38.756166,2017-09-22 00:00:00.000,2017-09-22,2019-07-13,2018-05-09 00:00:00,R11,SurveyDate
1,AF118,Andido,Afar,9.400856,40.323826,1900-01-09 09:37:00.118,2015-09-30,2020-06-03,2018-04-30 00:00:00,R11,SurveyDate
2,AF128,Bada Admorug,Afar,14.519210,40.116068,2016-06-10 00:00:00.000,2016-06-10,2023-08-05,2018-04-25 00:00:00,R11,SurveyDate
3,AF130,Gera,Afar,12.059997,41.925306,2015-01-20 00:00:00.000,2015-01-20,2023-08-30,2018-05-08 00:00:00,R11,SurveyDate
4,AF131d,Sodomta Drought,Afar,12.823153,40.989297,2014-04-21 00:00:00.000,2014-04-21,2021-08-24,2018-04-23 00:00:00,R11,SurveyDate
...,...,...,...,...,...,...,...,...,...,...,...
673711,TG1095,Laelay Adi Gebaro,Tigray,14.141199,38.177646,2020-12-31,2020-12-31,2023-09-02,NaN,R33,DispReason
673712,TG1096,May Seye,Tigray,14.108307,38.138678,2020-12-31,2020-12-31,2023-09-02,NaN,R33,DispReason
673713,TG1086,Simret,Tigray,14.224869,38.852089,2023-07-02,2023-07-02,2023-09-02,NaN,R33,DispReason
673714,TG1097,Geter Zeban Gedena,Tigray,14.326730,37.889691,2023-08-25,2023-08-25,2023-09-02,NaN,R33,DispReason


In [274]:
# pivot fields into columns using the fields column
tidy = melted.pivot_table(
    index=base_columns + ["survey"],
    columns="field",
    values="value",
    aggfunc="first"
).reset_index()

In [275]:
# Rename lat/lon for clarity
if "meanLat" in tidy.columns: tidy = tidy.rename(columns={"meanLat": "lat"})
if "meanLong" in tidy.columns: tidy = tidy.rename(columns={"meanLong": "lon"})

# Parse dates (handles messy strings reasonably; unparseable -> NaT)
if "SurveyDate" in tidy.columns:
    tidy["survey_date"] = pd.to_datetime(tidy["SurveyDate"], errors="coerce")
    tidy = tidy.drop(columns=["SurveyDate"])


tidy['min_open_date'] = pd.to_datetime(tidy["minOpenDate"], errors = "coerce")
tidy['max_open_date'] = pd.to_datetime(tidy['maxOpenDate'], errors = "coerce")
tidy["OpenDate"] = tidy["min_open_date"].combine_first(tidy["max_open_date"])
tidy = tidy.drop(columns = ['minOpenDate', 'maxOpenDate','min_open_date', 'max_open_date'])


if "maxSurveyDate" in tidy.columns:
    tidy['CloseDate'] = pd.to_datetime(tidy["maxSurveyDate"], errors = "coerce")
    tidy = tidy.drop(columns = ['maxSurveyDate'])    

# Numeric population + households
for col in ["TotIndivd", "TotalHH"]:
    if col in tidy.columns:
        tidy[col] = pd.to_numeric(tidy[col], errors="coerce")

In [276]:
# rename the field column to the OID column
tidy = tidy.rename(columns={"TotIndivd": "TotPop", "TotalHH": "TotHH", "survey": "SurveyRound", "meanLat": "lat", "meanLong": "lon","survey_date": 'SurveyDate'})

# rearrange the columns in the dataframe
tidy = tidy[["SiteID", "SiteName", 'RegionName','OpenDate', 'CloseDate','SurveyRound', 'SurveyDate', 'SiteType','IsSiteOpen', 'TotPop', 'TotHH','DispReason', 'lat', 'lon']]

In [277]:
# Optional: sort for convenience
tidy = tidy.sort_values(["SiteID", "SurveyDate"])

In [278]:
tidy

field,SiteID,SiteName,RegionName,OpenDate,CloseDate,SurveyRound,SurveyDate,SiteType,IsSiteOpen,TotPop,TotHH,DispReason,lat,lon
0,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R11,2018-05-09,Collective Center,Yes,6355.0,1816.0,Conflict,8.980229,38.756166
1,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R12,2018-07-22,Collective Center,Yes,6490.0,1854.0,Conflict,8.980229,38.756166
2,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R13,2018-09-18,Collective Center,Yes,6490.0,1854.0,Conflict,8.980229,38.756166
3,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R14,2018-11-20,Collective Center,Yes,6583.0,1854.0,Conflict,8.980229,38.756166
4,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R15,2019-01-05,Collective Settlement/Centre,Yes,7370.0,1854.0,Conflict,8.980229,38.756166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32183,TG994,Metkel Limat,Tigray,2020-12-15,2023-08-28,R33,2023-06-18,Host community/families,Yes,80.0,37.0,Conflict,13.230100,39.168500
32184,TG995,Hadinet,Tigray,2021-03-19,2023-08-25,R33,2023-06-19,Host community/families,Yes,469.0,149.0,Conflict,13.139300,39.107800
32185,TG996,Dekera,Tigray,2020-12-15,2023-08-28,R33,2023-06-19,Host community/families,Yes,159.0,53.0,Conflict,13.211600,39.112300
32186,TG997,Wemberta Adeke Ala,Tigray,2020-12-20,2023-08-21,R33,2023-06-22,Host community/families,Yes,307.0,92.0,Conflict,13.325500,39.283900


In [279]:
# export to csv for Morgan
tidy.to_csv(data_processed/"idp_sites_long.csv", index=False)

In [280]:
# Drop rows with no date or no population (for animation)
if "SurveyDate" in tidy.columns and "TotPop" in tidy.columns:
    tidy = tidy.dropna(subset=["SurveyDate", "TotPop"])

In [281]:
tidy

field,SiteID,SiteName,RegionName,OpenDate,CloseDate,SurveyRound,SurveyDate,SiteType,IsSiteOpen,TotPop,TotHH,DispReason,lat,lon
0,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R11,2018-05-09,Collective Center,Yes,6355.0,1816.0,Conflict,8.980229,38.756166
1,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R12,2018-07-22,Collective Center,Yes,6490.0,1854.0,Conflict,8.980229,38.756166
2,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R13,2018-09-18,Collective Center,Yes,6490.0,1854.0,Conflict,8.980229,38.756166
3,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R14,2018-11-20,Collective Center,Yes,6583.0,1854.0,Conflict,8.980229,38.756166
4,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R15,2019-01-05,Collective Settlement/Centre,Yes,7370.0,1854.0,Conflict,8.980229,38.756166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32183,TG994,Metkel Limat,Tigray,2020-12-15,2023-08-28,R33,2023-06-18,Host community/families,Yes,80.0,37.0,Conflict,13.230100,39.168500
32184,TG995,Hadinet,Tigray,2021-03-19,2023-08-25,R33,2023-06-19,Host community/families,Yes,469.0,149.0,Conflict,13.139300,39.107800
32185,TG996,Dekera,Tigray,2020-12-15,2023-08-28,R33,2023-06-19,Host community/families,Yes,159.0,53.0,Conflict,13.211600,39.112300
32186,TG997,Wemberta Adeke Ala,Tigray,2020-12-20,2023-08-21,R33,2023-06-22,Host community/families,Yes,307.0,92.0,Conflict,13.325500,39.283900


### Convert to GeoPandas GeoDataFrame and Export as a GeoJSON

In [282]:
# convert to geopandas geodataframe
camps = gpd.GeoDataFrame(tidy,
                         geometry = gpd.points_from_xy(tidy["lat"], tidy["lon"]),
                         crs = "EPSG:4326").drop(columns = ['lat', 'lon'])

In [283]:
# view the first row of the dataframe
camps.head(1)

field,SiteID,SiteName,RegionName,OpenDate,CloseDate,SurveyRound,SurveyDate,SiteType,IsSiteOpen,TotPop,TotHH,DispReason,geometry
0,AA101,Gotara,Addis Ababa,2017-09-22,2019-07-13,R11,2018-05-09,Collective Center,Yes,6355.0,1816.0,Conflict,POINT (8.98023 38.75617)


In [284]:
# export as a geojson
camps.to_file(data_processed/"idp_sites_lon_wgs1984.geojson", driver = "GeoJSON")